# Импорт Библиотек для работы

In [ ]:
import numpy as np
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt




# Импорт библиотек для данных и основные пути

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

TRAIN_DATA_PATH = "train.csv"
TEST_DATA_PATH = "test.csv"


# Нормализация

In [ ]:
class DataNormalizer:
    """
    Класс для стандартизации числовых признаков.
    Преобразует данные к нулевому среднему и единичной дисперсии.
    """
    def __init__(self):
        self.mean_values = None
        self.std_values = None

    def calculate_statistics(self, data_matrix):
        self.mean_values = np.mean(data_matrix, axis=0)
        self.std_values = np.std(data_matrix, axis=0)
        # Защита от деления на ноль
        self.std_values[self.std_values == 0] = 1.0
        return self
    def apply_normalization(self, data_matrix):
        return (data_matrix - self.mean_values) / self.std_values
    
    def fit_and_transform(self, data_matrix):
        self.calculate_statistics(data_matrix)
        return self.apply_normalization(data_matrix)


# линейная регрессия

In [ ]:
class RegularizedLinearModel:
    """
    Реализация линейной регрессии с L2-регуляризацией (Ridge).
    Используется для борьбы с переобучением.
    """
    def __init__(self, regularization_strength=1.0):
        self.reg_param = regularization_strength
        self.feature_weights = None
        self.constant_term = None
    
    def train_model(self, feature_matrix, target_vector):
        # Добавляем столбец для свободного члена
        design_matrix = np.c_[np.ones((feature_matrix.shape[0], 1)), feature_matrix]
        num_parameters = design_matrix.shape[1]
        identity_matrix = np.eye(num_parameters)
        identity_matrix[0, 0] = 0  # Не регуляризуем свободный член
        
        # Решение нормального уравнения с регуляризацией
        XtX = design_matrix.T.dot(design_matrix)
        Xty = design_matrix.T.dot(target_vector)
        regularized_XtX = XtX + self.reg_param * identity_matrix
        
        try:
            self.model_parameters = np.linalg.solve(regularized_XtX, Xty)
        except np.linalg.LinAlgError:
            self.model_parameters = np.linalg.pinv(regularized_XtX).dot(Xty)
            
        self.constant_term = self.model_parameters[0]
        self.feature_weights = self.model_parameters[1:]
        
    def generate_predictions(self, feature_matrix):
        return feature_matrix.dot(self.feature_weights) + self.constant_term


# Преобразование признаков

In [ ]:
def scale_to_unit_range(series):
    """
    Масштабирование признака к диапазону [0, 1].
    """
    return (series - series.min()) / (series.max() - series.min())

def process_temporal_features(dataframe):
    """
    Извлечение информации из временных меток.
    Преобразует дату в числовые признаки.
    """
    df_processed = dataframe.copy()
    if 'ApplicationDate' in df_processed.columns:
        df_processed['ApplicationDate'] = pd.to_datetime(df_processed['ApplicationDate'], errors='coerce')
        
        most_common = df_processed['ApplicationDate'].mode()[0]
        df_processed['ApplicationDate'] = df_processed['ApplicationDate'].fillna(most_common)
        
        reference_date = pd.Timestamp("1970-01-01")
        df_processed["DaysSinceReference"] = (df_processed["ApplicationDate"] - reference_date).dt.days
        df_processed["YearOfApplication"] = df_processed["ApplicationDate"].dt.year
        df_processed["MonthOfApplication"] = df_processed["ApplicationDate"].dt.month
        
        df_processed = df_processed.drop(columns=['ApplicationDate'])
    return df_processed

def calculate_geometric_metrics(dataframe):
    """
    Создание геометрических метрик на основе ключевых финансовых показателей.
    """
    df_processed = dataframe.copy()
    
    credit_score = df_processed['CreditScore'].fillna(df_processed['CreditScore'].median())
    annual_income = df_processed['AnnualIncome'].fillna(df_processed['AnnualIncome'].median())
    debt_ratio = df_processed['DebtToIncomeRatio'].fillna(df_processed['DebtToIncomeRatio'].median())
    
    norm_score = scale_to_unit_range(credit_score)
    norm_income = scale_to_unit_range(annual_income)
    norm_debt = scale_to_unit_range(debt_ratio)
    
    df_processed['DistanceToRisk'] = np.sqrt(norm_score**2 + norm_income**2 + (1 - norm_debt)**2)
    df_processed['DistanceToStability'] = np.sqrt((1 - norm_score)**2 + (1 - norm_income)**2 + norm_debt**2)
    
    return df_processed

def engineer_base_features(dataframe):
    """
    Базовое преобразование признаков: заполнение пропусков,
    логарифмирование, создание отношений и бининг.
    """
    df_processed = dataframe.copy()
    
    # Заполнение пропущенных значений
    numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
    for column in numeric_columns:
        df_processed[column] = df_processed[column].fillna(df_processed[column].median())
        
    # Логарифмические преобразования
    columns_for_log = ["AnnualIncome", "TotalAssets", "TotalLiabilities", "NetWorth", "LoanAmount"]
    for column in columns_for_log:
        if column in df_processed.columns:
            minimum_value = df_processed[column].min()
            offset = abs(minimum_value) + 1.0 if minimum_value < 0 else 1.0
            df_processed[f'LogTransformed_{column}'] = np.log(df_processed[column] + offset)

    # Отношения между признаками
    if 'AnnualIncome' in df_processed.columns and 'NumberOfDependents' in df_processed.columns:
        df_processed['IncomePerMember'] = df_processed['AnnualIncome'] / (df_processed['NumberOfDependents'] + 1)
        
    if 'TotalLiabilities' in df_processed.columns and 'AnnualIncome' in df_processed.columns:
        df_processed['LiabilitiesToIncome'] = df_processed['TotalLiabilities'] / (df_processed['AnnualIncome'] + 10)
        
    if 'InterestRate' in df_processed.columns and 'BaseInterestRate' in df_processed.columns:
        df_processed['InterestDifference'] = df_processed['InterestRate'] - df_processed['BaseInterestRate']

    # Дискретизация непрерывных признаков
    if 'CreditScore' in df_processed.columns:
        df_processed['CreditScoreCategory'] = pd.qcut(df_processed['CreditScore'], q=10, labels=False, duplicates='drop')
    if 'AnnualIncome' in df_processed.columns:
        df_processed['IncomeCategory'] = pd.qcut(df_processed['AnnualIncome'], q=5, labels=False, duplicates='drop')
    if 'LoanAmount' in df_processed.columns:
        df_processed['LoanAmountCategory'] = pd.qcut(df_processed['LoanAmount'], q=10, labels=False, duplicates='drop')
    if 'DebtToIncomeRatio' in df_processed.columns:
        df_processed['DTICategory'] = pd.qcut(df_processed['DebtToIncomeRatio'], q=10, labels=False, duplicates='drop')
        
    return df_processed

def prepare_features_for_model(training_data, testing_data):
    """
    Основной пайплайн подготовки данных:
    1. Обработка временных признаков
    2. Создание базовых и геометрических признаков
    3. Target Encoding категориальных переменных
    4. Полиномиальные преобразования
    """
    train_df = training_data.copy()
    test_df = testing_data.copy()
    
    # Удаление экстремальных значений целевой переменной
    target_values = train_df['RiskScore']
    lower_bound = target_values.quantile(0.012)
    upper_bound = target_values.quantile(0.991)
    train_df = train_df[(train_df['RiskScore'] >= lower_bound) & (train_df['RiskScore'] <= upper_bound)].reset_index(drop=True)
    target_values = train_df['RiskScore']
    train_df = train_df.drop('RiskScore', axis=1)
    
    # Маркировка наборов данных
    train_df['dataset_type'] = 1
    test_df['dataset_type'] = 0
    combined_data = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)
    combined_data = combined_data.drop(columns=['ID', 'Id'], errors='ignore')
    
    print("   ...Извлечение временных признаков")
    combined_data = process_temporal_features(combined_data)
    
    print("   ...Создание базовых и геометрических признаков")
    combined_data = engineer_base_features(combined_data)
    combined_data = calculate_geometric_metrics(combined_data)
    
    # Кодирование категориальных признаков на основе целевой переменной
    categorical_features = [
        "EducationLevel", "EmploymentStatus", "HomeOwnershipStatus", "LoanPurpose", 
        "IncomeCategory", "CreditScoreCategory", "LoanAmountCategory", "DTICategory"
    ]
    categorical_features = [feature for feature in categorical_features if feature in combined_data.columns]
    
    training_subset = combined_data[combined_data['dataset_type'] == 1].copy()
    testing_subset = combined_data[combined_data['dataset_type'] == 0].copy()
    
    training_subset['RiskScore'] = target_values.values
    global_median = training_subset['RiskScore'].median()
    
    print("   ...Target Encoding категориальных признаков")
    for feature in categorical_features:
        encoding_map = training_subset.groupby(feature)['RiskScore'].median()
        combined_data.loc[combined_data['dataset_type'] == 1, f'{feature}_Encoded'] = training_subset[feature].map(encoding_map)
        combined_data.loc[combined_data['dataset_type'] == 0, f'{feature}_Encoded'] = testing_subset[feature].map(encoding_map)
        combined_data[f'{feature}_Encoded'] = combined_data[f'{feature}_Encoded'].fillna(global_median)
        
    # Удаление нечисловых столбцов
    non_numeric_columns = combined_data.select_dtypes(include=['object', 'category']).columns
    combined_data = combined_data.drop(columns=non_numeric_columns, errors='ignore')
    
    # Выделение числовых признаков
    combined_data = combined_data.select_dtypes(include=[np.number])
    
    X_train = combined_data[combined_data['dataset_type'] == 1].drop('dataset_type', axis=1).values
    X_test = combined_data[combined_data['dataset_type'] == 0].drop('dataset_type', axis=1).values
    
    # Удаление константных признаков
    print("   ...Удаление маловариативных признаков")
    variance_filter = VarianceThreshold(threshold=0)
    X_train = variance_filter.fit_transform(X_train)
    X_test = variance_filter.transform(X_test)
    
    # Полиномиальные преобразования признаков
    print("   ...Генерация полиномиальных признаков")
    
    feature_correlations = []
    for i in range(X_train.shape[1]):
        correlation = np.corrcoef(X_train[:, i], target_values)[0, 1]
        feature_correlations.append(abs(correlation) if not np.isnan(correlation) else 0)
        
    top_30_indices = np.argsort(feature_correlations)[-30:]
    poly_transformer_2 = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly2 = poly_transformer_2.fit_transform(X_train[:, top_30_indices])[:, 30:]
    X_test_poly2 = poly_transformer_2.transform(X_test[:, top_30_indices])[:, 30:]
    
    top_5_indices = np.argsort(feature_correlations)[-5:]
    poly_transformer_3 = PolynomialFeatures(degree=3, include_bias=False)
    X_train_poly3 = poly_transformer_3.fit_transform(X_train[:, top_5_indices])[:, 5:]
    X_test_poly3 = poly_transformer_3.transform(X_test[:, top_5_indices])[:, 5:]
    
    X_train_final = np.hstack([X_train, X_train_poly2, X_train_poly3])
    X_test_final = np.hstack([X_test, X_test_poly2, X_test_poly3])
    
    return X_train_final, target_values.values, X_test_final

# ВАЛИДАЦИЯ И ОЦЕНКА МОДЕЛИ

In [ ]:
def perform_cross_validation(feature_matrix, target_vector, regularization_param, folds=5):
    """
    Кросс-валидация для подбора оптимального параметра регуляризации.
    """
    sample_count = len(feature_matrix)
    fold_size = sample_count // folds
    indices = np.arange(sample_count)
    np.random.shuffle(indices)
    validation_scores = []
    
    for fold in range(folds):
        val_indices = indices[fold*fold_size : (fold+1)*fold_size]
        train_indices = np.setdiff1d(indices, val_indices)
        
        X_train_fold, X_val_fold = feature_matrix[train_indices], feature_matrix[val_indices]
        y_train_fold, y_val_fold = target_vector[train_indices], target_vector[val_indices]
        
        model = RegularizedLinearModel(regularization_param)
        model.train_model(X_train_fold, y_train_fold)
        predictions = model.generate_predictions(X_val_fold)
        validation_scores.append(np.mean((y_val_fold - predictions) ** 2))
        
    return np.mean(validation_scores)


# Запуск

In [ ]:
def execute_analysis():
    """
    Основная функция: загрузка данных, подготовка признаков,
    обучение модели и генерация предсказаний.
    """
    try:
        training_set = pd.read_csv(TRAIN_DATA_PATH)
        testing_set = pd.read_csv(TEST_DATA_PATH)
    except FileNotFoundError as e:
        print(f"Ошибка загрузки файлов: {e}")
        return

    print("🔧 Подготовка признаков...")
    X_train_processed, y_train_processed, X_test_processed = prepare_features_for_model(training_set, testing_set)
    print(f"✅ Общее количество признаков: {X_train_processed.shape[1]}")
    
    print("⚖️ Нормализация данных...")
    normalizer = DataNormalizer()
    X_train_normalized = normalizer.fit_and_transform(X_train_processed)
    X_test_normalized = normalizer.apply_normalization(X_test_processed)
    
    print("🔍 Подбор параметра регуляризации...")
    regularization_values = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0]
    best_cv_score = float('inf')
    optimal_parameter = 1.0
    
    for reg_value in regularization_values:
        cv_score = perform_cross_validation(X_train_normalized, y_train_processed, reg_value, folds=5)
        print(f"   Параметр: {reg_value:<6} | CV MSE: {cv_score:.4f}")
        if cv_score < best_cv_score:
            best_cv_score = cv_score
            optimal_parameter = reg_value
            
    print(f"🏆 Оптимальный параметр: {optimal_parameter}")
    
    final_model = RegularizedLinearModel(optimal_parameter)
    final_model.train_model(X_train_normalized, y_train_processed)
    
    train_predictions = final_model.generate_predictions(X_train_normalized)
    print(f"📊 MSE на обучающей выборке: {np.mean((y_train_processed - train_predictions) ** 2):.4f}")
    
    test_predictions = final_model.generate_predictions(X_test_normalized)
    # Ограничение значений предсказаний
    test_predictions = np.clip(test_predictions, 0, 100)
    
    result_dataframe = pd.DataFrame({
        'ID': testing_set['ID'] if 'ID' in testing_set.columns else range(len(test_predictions)),
        'RiskScore': test_predictions
    })
    result_dataframe.to_csv('prediction_results.csv', index=False)
    print("💾 Результаты сохранены: prediction_results.csv")
    
    # Визуализация распределения предсказаний
    sns.histplot(result_dataframe["RiskScore"], bins=30)
    plt.title('Распределение предсказанных значений RiskScore')
    plt.xlabel('RiskScore')
    plt.ylabel('Частота')
    plt.show()

    if __name__ == "__main__":
        execute_analysis()